# Building a recommender script using Turicreate library

Turi Create is an open-source python library developed by Apple, used to create core machine learning models for supervised and unsupervised learning. The models are used for classification, object detection, style transfers, regression, and recommender system by simplifying the development of custom machine learning models.

In this notebook, for each user we find their frequency of purchase for each article id they had purchased in the past.

Then, the top 10 articles are found for each user based on their frequency of purchase

This dataset is used to build the recommender system

In [ ]:
!pip install turicreate --user

In [ ]:
import os
import numpy as np
import pandas as pd
import gc

import time
import turicreate as tc
from sklearn.model_selection import train_test_split

import sys, warnings, time, os, copy, gc, re, random, pickle, cudf
warnings.filterwarnings('ignore')
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# pd.set_option('display.max_rows', 50)
# pd.set_option('display.max_columns', None)
# pd.set_option("display.max_colwidth", 10000)
import seaborn as sns
sns.set()
from pandas.io.json import json_normalize
from pprint import pprint
from pathlib import Path
from tqdm import tqdm
tqdm.pandas()
from collections import Counter
from datetime import datetime, timedelta


In [ ]:
#articles = cudf.read_csv("../input/h-and-m-personalized-fashion-recommendations/articles.csv")
#customers = cudf.read_csv("../input/h-and-m-personalized-fashion-recommendations/customers.csv")
transactions = cudf.read_csv("../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv")
#sample_sub = cudf.read_csv("../input/h-and-m-personalized-fashion-recommendations/sample_submission.csv")

In [ ]:
# Take recent 2 transactions only
#transactions = transactions.sort_values(['customer_id','article_id','t_dat'],ascending=[1,1,0])
#transactions = transactions[['customer_id','article_id']].to_pandas().groupby(['customer_id']).head(2)

In [ ]:
transactions.head()

In [ ]:
transactions = transactions[['customer_id','article_id']].groupby(['customer_id','article_id']).agg({'article_id':'count'})
#transactions_og = transactions.copy()

In [ ]:
#transactions_og.columns=['purchase_count']
#transactions_og.reset_index(inplace=True)

In [ ]:
transactions.columns=['purchase_count']
transactions.reset_index(inplace=True)
transactions.head()

In [ ]:
#def create_data_dummy(data):
#    data_dummy = data.copy()
#    data_dummy['purchase_dummy'] = 1
#    return data_dummy
#data_dummy = create_data_dummy(transactions)

In [ ]:
#transactions['tot_item_purcahsed'] = 
tot_transactions = transactions[['customer_id','article_id']].groupby(['customer_id']).agg({'article_id':'count'})

In [ ]:
tot_transactions.reset_index(inplace=True)

In [ ]:
tot_transactions.columns = ['customer_id','tot_purchase']

In [ ]:
transactions = cudf.merge(transactions,tot_transactions,on='customer_id')
transactions.head()

In [ ]:
transactions.columns = ['customer_id','article_id','purchase_count','tot_purchase']

In [ ]:
transactions['scaled_purchase_freq'] = transactions['purchase_count']/transactions['tot_purchase']


In [ ]:
transactions.drop(['purchase_count','tot_purchase'],axis=1,inplace=True)

In [ ]:
transactions = transactions.sort_values(['customer_id','article_id','scaled_purchase_freq'],ascending=[1,1,0])

In [ ]:
transactions = transactions.to_pandas().groupby(['customer_id']).head(10)

In [ ]:
transactions.shape

In [ ]:
del tot_transactions

# Building a recommender model using turicreate

In [ ]:
model = tc.item_similarity_recommender.create(tc.SFrame(transactions), 'customer_id', 'article_id', target='scaled_purchase_freq')

In [ ]:
# constant variables to define field names include:
user_id = 'customer_id'
item_id = 'article_id'
users_to_recommend = transactions[['customer_id']]['customer_id'].drop_duplicates().tolist()
n_rec = 12 # number of items to recommend
n_display = 12 # to display the first few rows in an output dataset

In [ ]:
recom = model.recommend(users=users_to_recommend, k=n_rec)

In [ ]:
df_rec = recom.to_dataframe()

In [ ]:
df_rec.shape

In [ ]:
len(df_rec['customer_id'].drop_duplicates())

In [ ]:
df_rec = df_rec[['customer_id','article_id']]

In [ ]:
df_rec['prediction'] = df_rec.groupby(['customer_id'])['article_id'].transform(lambda x: ' '.join('0'+x.astype(str)))

In [ ]:
df_rec.drop('article_id',inplace=True,axis=1)

In [ ]:
df_rec['prediction'][0]

In [ ]:
df_rec.drop_duplicates(inplace=True)

In [ ]:
df_rec.shape

# for remaining ids

In [ ]:
sub7 = pd.read_csv('../input/h-m-framework-for-partitioned-validation/submission.csv').sort_values('customer_id').reset_index(drop=True)

In [ ]:
sub7 = sub7[~sub7['customer_id'].isin(df_rec['customer_id'])]

In [ ]:
frames = [df_rec, sub7]
result = pd.concat(frames)

In [ ]:
result.shape

In [ ]:
result.to_csv('turicreate_top10_0.0232.csv',index=False)

# Done